In [7]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.prompts import PromptTemplate


prompt = PromptTemplate.from_template(
    "A {word} is a",
)

llm = HuggingFacePipeline.from_model_id(
    model_id="gpt2",
    task="text-generation",
    device=-1,  # 0[GPU], -1[CPU(Default)]
    pipeline_kwargs={
        "max_new_tokens": 150,
    },
)

chain = prompt | llm

chain.invoke({"word": "kimchi"})

Using pad_token, but it is not set yet.


ValueError: Got device==0, device is required to be within [-1, 0)